# Functions!

In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk
nltk.data.path.append("/home/romaric/code/nghia95/fake-data-detector/notebooks/roma_NTLK_Data_Cache")
from nltk.corpus import cmudict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from textblob import TextBlob
from gensim.models import LsiModel
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
import textstat


In [2]:

cmu_dict = cmudict.dict()  # This should load from the cache
print(cmu_dict["hello"])

[['HH', 'AH0', 'L', 'OW1'], ['HH', 'EH0', 'L', 'OW1']]


In [3]:
def basic_cleaning(text):
    if not isinstance(text, str):  # Convert to string if it's not
       text = str(text)
    # Remove whitespace
    prepoc_text = text.strip()
    # Lowercasing
    prepoc_text = prepoc_text.lower()
    # remove digits
    prepoc_text = "".join(char for char in prepoc_text if not char.isdigit())
    # remove punctuation
    for punctuation in string.punctuation:
        prepoc_text = prepoc_text.replace(punctuation," ")
    # remove regex
    prepoc_text = re.sub('<[^<]+?',"",prepoc_text)

    return prepoc_text

In [4]:
def cons_density(text):

    consonnant = sum(1 for char in text if char.isalpha() and char not in "aeiou")
    vowel = sum(1 for char in text if char.isalpha() and char in "aeiou")
    total_letters = vowel + consonnant
    return round((consonnant/(vowel + consonnant)),3) if total_letters > 0 else 0

In [5]:
def get_word_stress(word):
    cmu_dict = cmudict.dict()
    if word in cmu_dict:
        return sum(int(char) for syllable in cmu_dict[word][0] for char in syllable if char.isdigit())
    return 0

def get_sentence_stress(sentence):
    words = sentence.split()
    stress_values = [get_word_stress(word) for word in words]
    return sum(stress_values)

In [6]:
def redundance(text):
    # give a redundance score, considering the lenght of each text, if a lemmatized words appears more than three times the mean, it is considered redundant.

    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    clean_tokens = [w for w in tokens if w not in stop_words]

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(w) for w in clean_tokens]

    word_counts = Counter(lemmatized_tokens)
    mean_freq = sum(word_counts.values()) / len(word_counts) if len(word_counts)!= 0 else 0

    if mean_freq != 0:
        score = sum(1 for word, count in word_counts.items() if count > 2.5 * mean_freq)
    else:
        score = 0

    return score

In [7]:
def sentiment_polarity(text):
    sent_pol = TextBlob(text).sentiment.polarity
    return abs(round(sent_pol,3))

In [8]:
def word_choice(text):
    common_ai_words =["commendable",'transhumanist', 'meticulous', 'elevate','hello', 'tapestry','leverage',
                  'journey', 'headache','resonate','testament','explore', 'binary','delve',
                  'enrich', 'seamless','multifaceted', 'sorry','foster', 'convey', 'beacon',
                  'interplay', 'oh', 'navigate','form','adhere','cannot', 'landscape','remember',
                  'paramount', 'comprehensive', 'placeholder','grammar','real','summary','symphony',
                  'furthermore','relationship','ultimately','profound','art','supercharge','evolve',
                  'beyoud','reimagine','vibrant', 'robust','pivotal','certainly','quinoa','orchestrate','align',
                  'diverse','recommend','annals','note','employ','bustling','indeed','digital','enigma', 'outfit',
                  'indelible','refrain','culture','treat','emerge','meticulous','esteemed','weight','whimsical','bespoke',
                  'highlight','antagonist','unlock','key','breakdown','tailor','misinformation','treasure','paradigm','captivate',
                  'song','underscore','calculate','especially','climate','hedging','inclusive','exercise','ai','embrace',
                  'level','nuance','career','dynamic','accent','ethos','cheap','firstly','online','goodbye'
                  ]
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    word_count = 0
    for word in text.split():
        if word in common_ai_words:
            word_count += 1

    return word_count

In [9]:
def coherence(text):
    # uses gensim to measure coherence, use the lsi model(latent semantic indexing, coherence c_v because we provide the text)
    tokens = word_tokenize(text)
    if not tokens:
        coherence_score = 0
    else:
        dictionary = corpora.Dictionary([tokens])
        corpus_gensim = [dictionary.doc2bow(tokens)]
        lsa_model = LsiModel(corpus_gensim, id2word=dictionary)

        coherence_model = CoherenceModel(
            model=lsa_model,
            texts=[tokens],
            dictionary=dictionary,
            coherence='c_v'
        )
        coherence_score = coherence_model.get_coherence()
    return coherence_score

In [10]:
def reading_ease(text):
    reading_ease= textstat.flesch_reading_ease(text)
    return reading_ease


def gunning_fog(text):
    gunning_fog = textstat.gunning_fog(text)
    return gunning_fog

In [11]:
text = "Hello is the cat cat cat name, it is commendable. The cat eat the cat fish that was in the bowl of the cat, the cat is a bad cat!"
print(f'cleaned :{basic_cleaning(text)}')
print(f'consonnance density :{cons_density(text)}')
print(f'stress value :{get_sentence_stress(text)}')
print(f'redundance :{redundance(text)}')
print(f'sentiment :{sentiment_polarity(text)}')
print(f'unusual word count :{word_choice(text)}')
print(f'coherence :{coherence(text)}')
print(f'reading ease :{reading_ease(text)}')
print(f'gunning fog :{gunning_fog(text)}')

cleaned :hello is the cat cat cat name  it is commendable  the cat eat the cat fish that was in the bowl of the cat  the cat is a bad cat 
consonnance density :0.625
stress value :17
redundance :1
sentiment :0.875
unusual word count :2
coherence :0.9999999999999998
reading ease :98.55
gunning fog :7.33


# Create Pipeline
We want to add columns, not transform them ==> no ColumnTransformer <br>
Function transformer?<br>
But firt we need to get our preprocessed data...

In [12]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

In [13]:
data_load = pd.read_csv("/home/romaric/code/nghia95/fake-data-detector/data/1k_sampled_dataset.csv")
data = data_load.copy()
data.head()

,text,source,prompt_id,text_length,word_count
0,The Philosophy and Ethics of Transhumanism\n\n...,GPT-3.5,1920,2558,394
1,Crime-tracking app Citizen is launching its ow...,Flan-T5-XXL,0,378,62
2,The court in Novorossiysk gave two of the danc...,GLM-130B,0,621,109
3,"then drops the drumsticks, poses, then walks o...",GPT-J,0,513,90
4,On tally went to the beach. She found a sand d...,GPT-J,0,4984,846


In [14]:
data["AI_gen"] = data["source"].apply(lambda x: 0 if x == "Human" else 1)

In [15]:
X=pd.DataFrame(data["text"][:3])
y=data["AI_gen"]

## Test No parallelism

In [16]:
class InputHandler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if isinstance(X, str):
            X = [X]
        return pd.DataFrame({"text": X})

class TextPreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["preprocessed"]

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            X = X["text"]
        cleaned = X.apply(basic_cleaning)
        return pd.DataFrame({"preprocessed": cleaned})

class ConsDensity(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["cons_density"]

    def transform(self, X):
        return X["preprocessed"].apply(cons_density).values.reshape(-1, 1)

class Stress(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["stress_value"]

    def transform(self, X):
        return X["preprocessed"].apply(get_sentence_stress).values.reshape(-1, 1)

class Sentiment(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["sentiment_score"]

    def transform(self, X):
        return X["preprocessed"].apply(sentiment_polarity).values.reshape(-1, 1)

class Redundance(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["redundance"]

    def transform(self, X):
        return X["preprocessed"].apply(redundance).values.reshape(-1, 1)

class UnusualWord(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["unusual_words"]

    def transform(self, X):
        return X["preprocessed"].apply(word_choice).values.reshape(-1, 1)

class Coherence(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["coherence"]

    def transform(self, X):
        return X["preprocessed"].apply(coherence).values.reshape(-1, 1)

class ReadingEase(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["reading_ease"]

    def transform(self, X):
        return X["text"].apply(reading_ease).values.reshape(-1, 1)

class GunningFog(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["gunning_fog"]

    def transform(self, X):
        return X["text"].apply(gunning_fog).values.reshape(-1, 1)


In [17]:
pipeline = Pipeline([
    ("input_handler", InputHandler()),
    ("union", FeatureUnion([
        ("preprocessed_features", Pipeline([
            ("preprocessor", TextPreprocessor()),
            ("features", FeatureUnion([
                ("cons_density", ConsDensity()),
                ("stress_value", Stress()),
                ("sentiment_score", Sentiment()),
                ("redundance", Redundance()),
                ("unusualword", UnusualWord()),
                ("coherence", Coherence())
            ]))
        ])),
        ("original_text_features", FeatureUnion([
            ("readingease", ReadingEase()),
            ("gunningfog", GunningFog())
        ]))
    ]))
])


feature_names = [
    "cons_density", "stress_value", "sentiment_score",
    "redundance", "unusual_words", "coherence",
    "reading_ease", "gunning_fog"
]

In [18]:
pipeline

Pipeline(steps=[('input_handler', InputHandler()),
                ('union',
                 FeatureUnion(transformer_list=[('preprocessed_features',
                                                 Pipeline(steps=[('preprocessor',
                                                                  TextPreprocessor()),
                                                                 ('features',
                                                                  FeatureUnion(transformer_list=[('cons_density',
                                                                                                  ConsDensity()),
                                                                                                 ('stress_value',
                                                                                                  Stress()),
                                                                                                 ('sentiment_score',
                                                                                                  Sentiment()),
                                                                                                 ('redundance',
                                                                                                  Redundance()),
                                                                                                 ('unusualword',
                                                                                                  UnusualWord()),
                                                                                                 ('coherence',
                                                                                                  Coherence())]))])),
                                                ('original_text_features',
                                                 FeatureUnion(transformer_list=[('readingease',
                                                                                 ReadingEase()),
                                                                                ('gunningfog',
                                                                                 GunningFog())]))]))])

In [19]:
type(text)

str

In [20]:
X = text
X

'Hello is the cat cat cat name, it is commendable. The cat eat the cat fish that was in the bowl of the cat, the cat is a bad cat!'

In [22]:
X_processed = pipeline.fit_transform(X)
X_processed_df = pd.DataFrame(X_processed, columns=feature_names)

In [23]:
X_processed_df

,cons_density,stress_value,sentiment_score,redundance,unusual_words,coherence,reading_ease,gunning_fog
0,0.625,22.0,0.7,1.0,2.0,1.0,98.55,7.33


In [24]:
X_processed_df.shape

(1, 8)

## With parallelism

In [25]:
class InputHandler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if isinstance(X, str):
            X = [X]
        return pd.DataFrame({"text": X})

class TextPreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["preprocessed"]

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            X = X["text"]
        cleaned = X.apply(basic_cleaning)
        return pd.DataFrame({"preprocessed": cleaned})

class ConsDensity(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["cons_density"]

    def transform(self, X):
        return X["preprocessed"].apply(cons_density).values.reshape(-1, 1)

class Stress(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["stress_value"]

    def transform(self, X):
        return X["preprocessed"].apply(get_sentence_stress).values.reshape(-1, 1)

class Sentiment(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["sentiment_score"]

    def transform(self, X):
        return X["preprocessed"].apply(sentiment_polarity).values.reshape(-1, 1)

class Redundance(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["redundance"]

    def transform(self, X):
        return X["preprocessed"].apply(redundance).values.reshape(-1, 1)

class UnusualWord(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["unusual_words"]

    def transform(self, X):
        return X["preprocessed"].apply(word_choice).values.reshape(-1, 1)

class Coherence(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["coherence"]

    def transform(self, X):
        return X["preprocessed"].apply(coherence).values.reshape(-1, 1)

class ReadingEase(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["reading_ease"]

    def transform(self, X):
        return X["text"].apply(reading_ease).values.reshape(-1, 1)

class GunningFog(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["gunning_fog"]

    def transform(self, X):
        return X["text"].apply(gunning_fog).values.reshape(-1, 1)


In [26]:
pipeline = Pipeline([
    ("input_handler", InputHandler()),
    ("union", FeatureUnion([
        ("preprocessed_features", Pipeline([
            ("preprocessor", TextPreprocessor()),
            ("features", FeatureUnion([
                ("cons_density", ConsDensity()),
                ("stress_value", Stress()),
                ("sentiment_score", Sentiment()),
                ("redundance", Redundance()),
                ("unusualword", UnusualWord()),
                ("coherence", Coherence())
            ]))
        ])),
        ("original_text_features", FeatureUnion([
            ("readingease", ReadingEase()),
            ("gunningfog", GunningFog())
        ]))
    ], n_jobs=-1))
])


feature_names = [
    "cons_density", "stress_value", "sentiment_score",
    "redundance", "unusual_words", "coherence",
    "reading_ease", "gunning_fog"
]

In [27]:
pipeline

Pipeline(steps=[('input_handler', InputHandler()),
                ('union',
                 FeatureUnion(n_jobs=-1,
                              transformer_list=[('preprocessed_features',
                                                 Pipeline(steps=[('preprocessor',
                                                                  TextPreprocessor()),
                                                                 ('features',
                                                                  FeatureUnion(transformer_list=[('cons_density',
                                                                                                  ConsDensity()),
                                                                                                 ('stress_value',
                                                                                                  Stress()),
                                                                                                 ('sentiment_score',
                                                                                                  Sentiment()),
                                                                                                 ('redundance',
                                                                                                  Redundance()),
                                                                                                 ('unusualword',
                                                                                                  UnusualWord()),
                                                                                                 ('coherence',
                                                                                                  Coherence())]))])),
                                                ('original_text_features',
                                                 FeatureUnion(transformer_list=[('readingease',
                                                                                 ReadingEase()),
                                                                                ('gunningfog',
                                                                                 GunningFog())]))]))])

In [28]:
X

'Hello is the cat cat cat name, it is commendable. The cat eat the cat fish that was in the bowl of the cat, the cat is a bad cat!'

In [30]:
X_processed = pipeline.fit_transform(X)
X_processed_df = pd.DataFrame(X_processed, columns=feature_names)

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
processed_df.shape

(1, 8)

### concat and check

In [ ]:
X_processed_df = pd.DataFrame(X_processed)
X_processed_df.head()

,0,1,2,3,4,5,6,7
0,25.08,14.57,0.599,380.0,0.066,4.0,1.0,0.177397
1,50.43,7.54,0.613,64.0,0.167,2.0,0.0,1.000000
2,66.27,12.76,0.622,94.0,0.087,1.0,0.0,1.000000


In [ ]:
X_processed_df.shape

(3, 8)

In [ ]:
X_final = pd.concat([X, X_processed_df], axis=1)
X_final.shape

(3, 9)